## Kernel: Python 3.8

In [1]:
from dotenv import dotenv_values
from datetime import timedelta, datetime

import pandas as pd
import requests
import locale

## Get Authority

In [ ]:
config = dotenv_values(".env") # get values from .env file
locale.setlocale(locale.LC_ALL, "id") # set local language

def login() -> str:
    try:
        url = 'https://api.kampusmerdeka.kemdikbud.go.id/user/auth/login/mbkm'
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
        },
        
    except:
    return: